In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path

data_folder = Path("../../data")

prod_data_folder: Path = data_folder.joinpath("30_prod/")
model_artifact = Path("../../models/artifacts")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({"figure.figsize": (8, 6), "text.usetex": True})

GENERATOR_KEYWORD: str = "GENERATOR"
DISCRIMINATOR_KEYWORD: str = "DISCRIMINATOR"

MODEL_TYPE: str = "GAN_LSTM"

AAPL_TICKER: str = "AAPL"
ACWI_TICKER: str = "ACWI"
JPM_TICKER: str = "JPM"

aapl_stock_filename: str = f"{AAPL_TICKER}.csv"
acwi_stock_filename: str = f"{ACWI_TICKER}.csv"
jpm_stock_filename: str = f"{JPM_TICKER}.csv"

aapl_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(aapl_stock_filename))
acwi_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(acwi_stock_filename))
jpm_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(jpm_stock_filename))

aapl_df["Date"] = pd.to_datetime(aapl_df["Date"], format="%Y-%m-%d")
acwi_df["Date"] = pd.to_datetime(acwi_df["Date"], format="%Y-%m-%d")
jpm_df["Date"] = pd.to_datetime(jpm_df["Date"], format="%Y-%m-%d")

aapl_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
acwi_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
jpm_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

aapl_min_max_scaler.fit(aapl_df[["Close"]])
acwi_min_max_scaler.fit(acwi_df[["Close"]])
jpm_min_max_scaler.fit(jpm_df[["Close"]])

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, seq_length=60, n_features=1):
        super().__init__()
        self.hidden_dim = 128
        self.seq_length = seq_length
        self.n_features = n_features

        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            nn.Linear(256, self.hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(self.hidden_dim),
        )

        self.lstm = nn.LSTM(
            input_size=self.hidden_dim,
            hidden_size=self.hidden_dim,
            num_layers=2,
            batch_first=True,
        )

        self.output_layer = nn.Sequential(
            nn.Linear(self.hidden_dim, n_features), nn.Tanh()
        )

    def forward(self, z):
        x = self.fc(z)
        x = x.unsqueeze(1).repeat(1, self.seq_length, self.n_features)
        lstm_out, _ = self.lstm(x)
        return self.output_layer(lstm_out)


class Discriminator(nn.Module):
    def __init__(self, n_features=1):
        super().__init__()
        self.hidden_dim = 64

        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=self.hidden_dim,
            num_layers=2,
            batch_first=True,
        )

        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_dim, 32),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_output = lstm_out[:, -1, :]
        return self.classifier(last_output)

### Apple

In [ ]:
aapl_generator_model_filename: str = (
    f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{AAPL_TICKER}.pth"
)
aapl_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{AAPL_TICKER}.pth"
)

aapl_generator = Generator(100)
aapl_discriminator = Discriminator()

aapl_generator.load_state_dict(
    torch.load(model_artifact / aapl_generator_model_filename)
)
aapl_discriminator.load_state_dict(
    torch.load(model_artifact / aapl_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

aapl_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(100, 1, 100)
    for i in range(100):
        output_tensor = aapl_generator(input_tensor[i])
        ax.plot(
            aapl_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("AAPL RNN Generator Model Architecture")
plt.show()

### ACWI

In [ ]:
acwi_generator_model_filename: str = (
    f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{ACWI_TICKER}.pth"
)
acwi_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{ACWI_TICKER}.pth"
)

acwi_generator = Generator(100)
acwi_discriminator = Discriminator()

acwi_generator.load_state_dict(
    torch.load(model_artifact / acwi_generator_model_filename)
)
acwi_discriminator.load_state_dict(
    torch.load(model_artifact / acwi_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

acwi_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(100, 1, 100)
    for i in range(100):
        output_tensor = acwi_generator(input_tensor[i])
        ax.plot(
            acwi_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("ACWI RNN Generator Model Architecture")
plt.show()

### JPM

In [ ]:
jpm_generator_model_filename: str = f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{JPM_TICKER}.pth"
jpm_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{JPM_TICKER}.pth"
)

jpm_generator = Generator(100)
jpm_discriminator = Discriminator()

jpm_generator.load_state_dict(torch.load(model_artifact / jpm_generator_model_filename))
jpm_discriminator.load_state_dict(
    torch.load(model_artifact / jpm_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

jpm_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(40, 1, 100)
    for i in range(40):
        output_tensor = jpm_generator(input_tensor[i])
        ax.plot(
            jpm_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("JPM RNN Generator Model Architecture")
plt.show()